In [4]:
import cv2
import imutils
import numpy as np
import pygame
import dlib
from scipy.spatial import distance
from imutils import face_utils
from IPython.display import display, clear_output

# Initialize pygame mixer for sound alerts
pygame.mixer.init()
alert_sound = pygame.mixer.Sound("alert.wav")  # Load a sound file (replace with your file)

# Initialize dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# Function to calculate Mouth Aspect Ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[1], mouth[7])
    B = distance.euclidean(mouth[3], mouth[5])
    C = distance.euclidean(mouth[0], mouth[4])
    return (A + B) / (2.0 * C)

# Thresholds for detecting drowsiness and yawning
thresh_ear = 0.25
thresh_mar = 0.6
frame_check = 20
flag = 0

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces using dlib's detector
    faces = detector(gray, 0)
    
    for face in faces:
        # Get the facial landmarks using dlib's shape predictor
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        
        # Extract the left/right eyes and mouth using landmark indices
        (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
        (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]
        (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["mouth"]

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        mouth = shape[mStart:mEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        mar = mouth_aspect_ratio(mouth)
        
        # Draw contours for visualization
        cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 0, 255), 1)
        
        # Drowsiness Alert: Check if EAR is below the threshold
        if ear < thresh_ear:
            flag += 1
            if flag >= frame_check:
                cv2.putText(frame, "****************ALERT!****************", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                alert_sound.play()  # Play alert sound
        else:
            flag = 0
        
        # Yawning Alert: Check if MAR is above the threshold
        if mar > thresh_mar:
            cv2.putText(frame, "Yawning!", (200, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            alert_sound.play()
            
        # Display EAR and MAR values on the frame
        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, f"MAR: {mar:.2f}", (300, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Display the frame (for Jupyter Notebook)
    clear_output(wait=True)
    display(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'dlib'

In [8]:
!pip install dlib


  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for dlib
Failed to build dlib


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [47 lines of output]
  C:\Users\souna\AppData\Local\Temp\pip-install-aurdi3g0\dlib_7d520b48c3454ed685d995e0dff6c1d2\setup.py:234: SyntaxWarning: invalid escape sequence '\('
    major = re.findall("set\(CPACK_PACKAGE_VERSION_MAJOR.*\"(.*)\"", open(cmake_file).read())[0]
  C:\Users\souna\AppData\Local\Temp\pip-install-aurdi3g0\dlib_7d520b48c3454ed685d995e0dff6c1d2\setup.py:235: SyntaxWarning: invalid escape sequence '\('
    minor = re.findall("set\(CPACK_PACKAGE_VERSION_MINOR.*\"(.*)\"", open(cmake_file).read())[0]
  C:\Users\souna\AppData\Local\Temp\pip-install-aurdi3g0\dlib_7d520b48c3454ed685d995e0dff6c1d2\setup.py:236: SyntaxWarning: invalid escape sequence '\('
    patch = re.findall("set\(CPACK_PACKAGE_VERSION_PATCH.*\"(.*)\"", open(cmake_file).read())[0]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on y